# Linear Solver Performance: Constant Coefficient Poisson, Single Core

### Part 1, Benchmark Setup and Execution

In [1]:
Console.WriteLine("Execution Date/time is " + DateTime.Now);

The below script needs to be able to find the current output cell; this is an easy method to get it.

Execution Date/time is 11/5/2021 6:08:59 AM


In [2]:
#r "BoSSSpad.dll"
//#r "../../../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [3]:
using BoSSS.Application.SipPoisson;

In [4]:
ExecutionQueues

index,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,AllowedDatabasesPaths
0,C:\BoSSStests,False,MSHPC-Gitrunner-HighPrio,dotnet,FDY\jenkinsci,DC2,[ fdygitrunner ],Highest,True,"[ { C:\BoSSStests == : LocalMountPath: C:\BoSSStests, PathAtRemote: }, { C:\Users\jenkinsci\ == : LocalMountPath: C:\Users\jenkinsci\, PathAtRemote: } ]"
1,C:\BoSSStests,False,MSHPC-Gitrunner-DefaultTest,dotnet,FDY\jenkinsci,DC2,[ fdygitrunner ],Normal,True,"[ { C:\BoSSStests == : LocalMountPath: C:\BoSSStests, PathAtRemote: } ]"
2,\\fdygitrunner\ValidationTests,False,MSHPC-AllNodes,dotnet,FDY\jenkinsci,DC2,"[ hpccluster, hpccluster2 ]",Normal,True,"[ { \\fdygitrunner\ValidationTests == : LocalMountPath: \\fdygitrunner\ValidationTests, PathAtRemote: } ]"


In [5]:
var myBatch = GetDefaultQueue();
myBatch

DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,AllowedDatabasesPaths
\\fdygitrunner\ValidationTests,False,MSHPC-AllNodes,dotnet,FDY\jenkinsci,DC2,"[ hpccluster, hpccluster2 ]",Normal,True,"[ { \\fdygitrunner\ValidationTests == : LocalMountPath: \\fdygitrunner\ValidationTests, PathAtRemote: } ]"


In [6]:
wmg.Init("LinslvPerf_ConstPoissonMpi1");
wmg.SetNameBasedSessionJobControlCorrelation();
wmg.AllJobs

Project name is set to 'LinslvPerf_ConstPoissonMpi1'.
Opening existing database '\\fdygitrunner\ValidationTests\LinslvPerf_ConstPoissonMpi1'.


## Utility definitions

In [7]:
static class Utils {
    // DOF per cell in 3D
    public static int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }    
    
    //Non-equidistant nodes
    public static double[] SinLinSpacing(double l, double r, double a, int n) {
        double[] linnodes = GenericBlas.Linspace(-Math.PI * 0.5, Math.PI * 0.5, n);
        double[] linnodes2 = GenericBlas.Linspace(-1, 1, n);
        double[] nodes = new double[n];

        for (int i = 0; i < n; i++)
            //nodes[i] = linnodes2[i] * (1 - a) + (1.0 - Math.Sin(linnodes[i])) * a;
            nodes[i] = linnodes2[i] * (1 - a) + Math.Sin(linnodes[i])*a;

        for (int i = 0; i < n; i++)
            nodes[i] = nodes[i] * (r - l)*0.5 + l;
        return nodes;
    }
}

## Init grids and save to database

In [8]:
wmg.Grids

Opening existing database 'C:\BoSSStests\LinslvPerf_ConstPoissonMpi1'.


#0: { Guid = a3bb3217-dcf0-4168-8e12-aabc3d069b0d; Name = LinslvPerf_ConstPoissonMpi1_J1310720; Cell Count = 1310720; Dim = 3 }
#1: { Guid = d5162e70-5b0e-45ed-9c45-7fad0ad49838; Name = LinslvPerf_ConstPoissonMpi1_J552960; Cell Count = 552960; Dim = 3 }
#2: { Guid = 85efeb60-e205-4709-b935-fa97e806dd83; Name = LinslvPerf_ConstPoissonMpi1_J163840; Cell Count = 163840; Dim = 3 }
#3: { Guid = 665339ec-13c0-4113-8aec-62ab05ba8df2; Name = LinslvPerf_ConstPoissonMpi1_J69120; Cell Count = 69120; Dim = 3 }
#4: { Guid = f3105a5d-29a0-410f-a614-cf347596514c; Name = LinslvPerf_ConstPoissonMpi1_J20480; Cell Count = 20480; Dim = 3 }
#5: { Guid = f6a0f14b-8c23-40af-b1c7-e765fb15acbe; Name = LinslvPerf_ConstPoissonMpi1_J2560; Cell Count = 2560; Dim = 3 }
#6: { Guid = 1e87dd13-d0ec-47a7-ae43-009dbd3e9781; Name = LinslvPerf_ConstPoissonMpi1_J320; Cell Count = 320; Dim = 3 }


Create meshes in various resolutions:
- domain $\Omega = (0,10) \times (-1,1) \times (-1,1)$; 
- a Dirichlet boundary is set for $x = 0$, everywhere else Neumann boundaries are assumed; 
  these are typically more challenging for a linear solver than Dirichlet boundaries.
- in order to make the problem a bit challenging, a non-equidistant spacing for $y$ and $z$ coordinate are used
- the domain shape is reminiscent of a typical grid for the simulation of channel flows, with exception of the boundary conditions

In [9]:
int[] Resolutions_3D = new int[] { 4, 8, 16, 24, 32, 48, 64 };
IGridInfo[] grids = new IGridInfo[Resolutions_3D.Length];
for(int cnt = 0; cnt < Resolutions_3D.Length; cnt++) {
    int Res = Resolutions_3D[cnt];    
    
    double[] xNodes = GenericBlas.Linspace(0, 10, Res*5 + 1);
    double[] yNodes = Utils.SinLinSpacing(-1, +1, 0.6, Res + 1);
    double[] zNodes = Utils.SinLinSpacing(-1, +1, 0.6, Res + 1);
    int J = (xNodes.Length - 1)*(yNodes.Length - 1)*(zNodes.Length - 1);
    
    string GridName = string.Format(wmg.CurrentProject + "_J" + J);
    
    //grids[cnt] = null;
    //foreach(IGridInfo grd in wmg.Grids) { 
    //    bool check = grd.Name.Contains(string.Format("_J"+ J));
    //    if(check) {
    //      grids[cnt] = grd;
    //    }
    //}
    
    grids[cnt] = wmg.Grids.SingleOrDefault(grd => grd.Name.Contains(string.Format("_J"+ J))); // check if an appropriate grid is already present in the database
    if(grids[cnt] == null) { // 
        Console.WriteLine($"Creating grid with {J} = {Res*5}x{Res}x{Res} cells.");
        
        GridCommons g;
        g      = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
        g.Name = GridName;
    
        g.DefineEdgeTags(delegate (double[] X) {
            if(Math.Abs(X[0] - 0.0) <= 1.0e-6)
                return BoundaryType.Dirichlet.ToString();
            else
                return BoundaryType.Neumann.ToString();
        });
      
        g = wmg.SaveGrid(g);  
        grids[cnt] = g;
    } else {
        Console.WriteLine("Found Grid: " + grids[cnt]);
        if(grids[cnt].NumberOfCells != J)
            throw new Exception("J mismatch");
        
        if(grids[cnt].SpatialDimension != 3)
            throw new Exception("D mismatch");
    }
}

Found Grid: { Guid = 1e87dd13-d0ec-47a7-ae43-009dbd3e9781; Name = LinslvPerf_ConstPoissonMpi1_J320; Cell Count = 320; Dim = 3 }
Found Grid: { Guid = f6a0f14b-8c23-40af-b1c7-e765fb15acbe; Name = LinslvPerf_ConstPoissonMpi1_J2560; Cell Count = 2560; Dim = 3 }
Found Grid: { Guid = f3105a5d-29a0-410f-a614-cf347596514c; Name = LinslvPerf_ConstPoissonMpi1_J20480; Cell Count = 20480; Dim = 3 }
Found Grid: { Guid = 665339ec-13c0-4113-8aec-62ab05ba8df2; Name = LinslvPerf_ConstPoissonMpi1_J69120; Cell Count = 69120; Dim = 3 }
Found Grid: { Guid = 85efeb60-e205-4709-b935-fa97e806dd83; Name = LinslvPerf_ConstPoissonMpi1_J163840; Cell Count = 163840; Dim = 3 }
Found Grid: { Guid = d5162e70-5b0e-45ed-9c45-7fad0ad49838; Name = LinslvPerf_ConstPoissonMpi1_J552960; Cell Count = 552960; Dim = 3 }
Found Grid: { Guid = a3bb3217-dcf0-4168-8e12-aabc3d069b0d; Name = LinslvPerf_ConstPoissonMpi1_J1310720; Cell Count = 1310720; Dim = 3 }


In [10]:
grids

#0: { Guid = 1e87dd13-d0ec-47a7-ae43-009dbd3e9781; Name = LinslvPerf_ConstPoissonMpi1_J320; Cell Count = 320; Dim = 3 }
#1: { Guid = f6a0f14b-8c23-40af-b1c7-e765fb15acbe; Name = LinslvPerf_ConstPoissonMpi1_J2560; Cell Count = 2560; Dim = 3 }
#2: { Guid = f3105a5d-29a0-410f-a614-cf347596514c; Name = LinslvPerf_ConstPoissonMpi1_J20480; Cell Count = 20480; Dim = 3 }
#3: { Guid = 665339ec-13c0-4113-8aec-62ab05ba8df2; Name = LinslvPerf_ConstPoissonMpi1_J69120; Cell Count = 69120; Dim = 3 }
#4: { Guid = 85efeb60-e205-4709-b935-fa97e806dd83; Name = LinslvPerf_ConstPoissonMpi1_J163840; Cell Count = 163840; Dim = 3 }
#5: { Guid = d5162e70-5b0e-45ed-9c45-7fad0ad49838; Name = LinslvPerf_ConstPoissonMpi1_J552960; Cell Count = 552960; Dim = 3 }
#6: { Guid = a3bb3217-dcf0-4168-8e12-aabc3d069b0d; Name = LinslvPerf_ConstPoissonMpi1_J1310720; Cell Count = 1310720; Dim = 3 }


## Setup Control Object for a Solver Run

### Values/Formulas for the Values of RHS and Boundary Conditions

In [11]:
string nl = System.Environment.NewLine;
string math_expressions_code = 
"static class InitialValues { " +
"    public static double RHS(double[] X) { " + nl +
"        return -Math.Sin(X[0]); " + nl +
"    } " + nl +
"     " + nl +
"    public static double DirichletBC(double[] X) { " + nl +
"        return 0.0; " + nl +
"    } " + nl +
"     " + nl +
"    public static double NeumannBC(double[] X) { " + nl +
"       if(Math.Abs(X[1] - 1.0) < 1.0e-8 || Math.Abs(X[1] + 1.0) < 1.0e-8) " + nl +
"           return 0; " + nl +
"       if(X.Length > 2 && (Math.Abs(X[2] - 1.0) < 1.0e-8 || Math.Abs(X[2] + 1.0) < 1.0e-8)) " + nl +
"           return 0; " + nl +
"     " + nl +
"       return Math.Cos(10.0); " + nl +
"   } " + nl +
"} " + nl;

In [12]:
var RHS_value = new Formula("InitialValues.RHS", false, math_expressions_code);
var NeumannBC_value = new Formula("InitialValues.NeumannBC", false, math_expressions_code);
var DirichletBC_value = new ConstantValue(0.0);

Brief test of the boundary values:

In [13]:
RHS_value.Evaluate(new Vector(0.2, 0.1), 0)

-0.19866933079506122

In [14]:
NeumannBC_value.Evaluate(new Vector(0.2, 0.1), 0)

-0.8390715290764524

### Setup of Parameter Study

Polynomial degrees to test:

In [15]:
//int[] PolyDegS = new int[] {2, 3, 4, 5, 6};
int[] PolyDegS = new int[] {2, 3, 5}; // reduced options for dev.

Solvers which we want to instrument:

In [16]:
using BoSSS.Solution.AdvancedSolvers;

In [17]:
LinearSolverCode[] solver_nameS = new LinearSolverCode[] {
    LinearSolverCode.classic_pardiso, 
    //LinearSolverCode.classic_mumps, 
    LinearSolverCode.exp_Kcycle_schwarz,
    LinearSolverCode.exp_gmres_levelpmg,
}; 

Maximum Dof for one calculation (we skip fine grids for higher polynomial orders):

In [18]:
int GetMaxAllowedDOF(LinearSolverCode code) {
    switch(code) {
        case LinearSolverCode.classic_pardiso:
        case LinearSolverCode.classic_mumps:
        return 1100000; // 1.1 Million for iterative solvers at maximum
    
        default: 
        return 5000000; // Up to 5 Million for iterative solvers
    }
}

Loop over all combinations of parameters and define a control object for each combo:

In [19]:
List<SipControl> controls = new List<SipControl>();
foreach(LinearSolverCode solver_name in solver_nameS) {
foreach(int k in PolyDegS) {   
foreach(IGridInfo grd in grids) {
    int Np = Utils.Np(k);
    int J  = grd.NumberOfCells;
    if(J*Np > GetMaxAllowedDOF(solver_name))
        continue;
    
    var ctrl = new SipControl();
    controls.Add(ctrl);
       
    
    string caseName = string.Format("J{0}_k{1}_{2}", J, k, solver_name);
    Console.WriteLine("setting up: " + caseName);

    ctrl.SetGrid(grd);
    ctrl.savetodb = true;
    ctrl.SetDGdegree(k);

    ctrl.LinearSolver.SolverCode           = solver_name;
    ctrl.LinearSolver.NoOfMultigridLevels  = int.MaxValue;
    ctrl.LinearSolver.TargetBlockSize      = Math.Min(J*Np-1,10000);
    ctrl.LinearSolver.ConvergenceCriterion = 1e-8;
    
    ctrl.InitialValues.Add("RHS", RHS_value);
    ctrl.AddBoundaryValue(BoundaryType.Dirichlet.ToString(), "T", DirichletBC_value);
    ctrl.AddBoundaryValue(BoundaryType.Neumann.ToString(), "T", NeumannBC_value);
    
    ctrl.SessionName = "SIP_" + caseName;
    
    ctrl.GridPartType = BoSSS.Foundation.Grid.GridPartType.METIS;
    //ctrl.GridPartType = BoSSS.Foundation.Grid.GridPartType.directHilbert;
}
}
}

setting up: J320_k2_classic_pardiso
setting up: J2560_k2_classic_pardiso
setting up: J20480_k2_classic_pardiso
setting up: J69120_k2_classic_pardiso
setting up: J320_k3_classic_pardiso
setting up: J2560_k3_classic_pardiso
setting up: J20480_k3_classic_pardiso
setting up: J320_k5_classic_pardiso
setting up: J2560_k5_classic_pardiso
setting up: J320_k2_exp_Kcycle_schwarz
setting up: J2560_k2_exp_Kcycle_schwarz
setting up: J20480_k2_exp_Kcycle_schwarz
setting up: J69120_k2_exp_Kcycle_schwarz
setting up: J163840_k2_exp_Kcycle_schwarz
setting up: J320_k3_exp_Kcycle_schwarz
setting up: J2560_k3_exp_Kcycle_schwarz
setting up: J20480_k3_exp_Kcycle_schwarz
setting up: J69120_k3_exp_Kcycle_schwarz
setting up: J163840_k3_exp_Kcycle_schwarz
setting up: J320_k5_exp_Kcycle_schwarz
setting up: J2560_k5_exp_Kcycle_schwarz
setting up: J20480_k5_exp_Kcycle_schwarz
setting up: J69120_k5_exp_Kcycle_schwarz
setting up: J320_k2_exp_gmres_levelpmg
setting up: J2560_k2_exp_gmres_levelpmg
setting up: J20480_k2

## Launch Jobs

In [20]:
foreach(var ctrl in controls) {
    Console.WriteLine(" Submitting: " + ctrl.SessionName);
    ctrl.RunBatch(myBatch); 
}

 Submitting: SIP_J320_k2_classic_pardiso
Info: Found successful session "LinslvPerf_ConstPoissonMpi1	SIP_J320_k2_classic_pardiso	11/04/2021 18:04:00	1f6cb5be..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: SIP_J2560_k2_classic_pardiso
Info: Found successful session "LinslvPerf_ConstPoissonMpi1	SIP_J2560_k2_classic_pardiso	11/04/2021 18:04:08	5ce0bf18..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: SIP_J20480_k2_classic_pardiso
Info: Found successful session "LinslvPerf_ConstPoissonMpi1	SIP_J20480_k2_classic_pardiso	11/04/2021 18:04:16	e814f336..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: SIP_J69120_k2_classic_pardiso
Info: Found successful session "LinslvPerf_ConstPoissonMpi1	SIP_J69120_k2_classic_pardiso	11/04/2021 18:04:27	a4a94934..." -- job is marked a

No submission, because job status is: FinishedSuccessful
 Submitting: SIP_J69120_k3_exp_gmres_levelpmg
Info: Found successful session "LinslvPerf_ConstPoissonMpi1	SIP_J69120_k3_exp_gmres_levelpmg	11/04/2021 18:08:57	372e3ccb..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: SIP_J163840_k3_exp_gmres_levelpmg
Note: Job was deployed (1) number of times, all failed.
Hint: want to re-activate the job.
No submission, because job status is: FailedOrCanceled
 Submitting: SIP_J320_k5_exp_gmres_levelpmg
Info: Found successful session "LinslvPerf_ConstPoissonMpi1	SIP_J320_k5_exp_gmres_levelpmg	11/04/2021 18:09:14	8e4754a4..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: SIP_J2560_k5_exp_gmres_levelpmg
Info: Found successful session "LinslvPerf_ConstPoissonMpi1	SIP_J2560_k5_exp_gmres_levelpmg	11/04/2021 18:09:26	5b7dbade..." -- job is marked as succ

In [21]:
//foreach(var ctrl in controls) {
//    Console.WriteLine(" Submitting: " + ctrl.SessionName); 
//    var aJob = new Job(ctrl.SessionName, typeof(SipPoissonMain));
//    aJob.SetControlObject(ctrl);
//    ((SlurmClient)myBatch).SlurmAccount = "project01217";  // Jens 
//    aJob.NumberOfMPIProcs = 1;
//    aJob.ExecutionTime    = "02:00:00";
//    //aJob.HHLR_project             = "project01217";
//    aJob.MemPerCPU                = "50000";
//    aJob.UseComputeNodesExclusive = true;
//    aJob.Activate(myBatch);
//}

In [22]:
/*
int cnt   = 0;
int iCore = -1;
foreach(var ctrl in controls) {

    if(cnt % JobsWithSameCore == 0)
        iCore++;
         
    Console.WriteLine(" Submitting: " + ctrl.SessionName); 
    var aJob = new Job(ctrl.SessionName, typeof(SipPoissonMain));
    aJob.SetControlObject(ctrl);
    ((SlurmClient)ExecutionQueues[1]).SlurmAccount = "project01217";  // Jens 
    aJob.NumberOfMPIProcs         = core_sweep[iCore];
    aJob.ExecutionTime            = "02:00:00";
    //aJob.HHLR_project             = "project01217";
    aJob.MemPerCPU                = "2500";
    aJob.UseComputeNodesExclusive = true;
    aJob.Activate(myBatch);
    cnt++;
}
*/

In [23]:
//foreach(var j in wmg.AllJobs.Values)
//    j.DeleteOldDeploymentsAndSessions();

### Wait for Completion and Check Job Status

In [24]:
wmg.BlockUntilAllJobsTerminate(3600*24*2); // wait at maximum two days for the jobs to finish

All jobs finished.


In [25]:
wmg.AllJobs

#0: SIP_J320_k2_classic_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)	SIP_J320_k2_classic_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#1: SIP_J2560_k2_classic_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)	SIP_J2560_k2_classic_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#2: SIP_J20480_k2_classic_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)	SIP_J20480_k2_classic_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#3: SIP_J69120_k2_classic_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)	SIP_J69120_k2_classic_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#4: SIP_J320_k3_classic_pardis

In [26]:
wmg.Sessions

#0: LinslvPerf_ConstPoissonMpi1	SIP_J69120_k3_exp_Kcycle_schwarz	11/04/2021 18:06:45	8666816b...
#1: LinslvPerf_ConstPoissonMpi1	SIP_J163840_k2_exp_Kcycle_schwarz	11/04/2021 18:06:01	f68b6d88...
#2: LinslvPerf_ConstPoissonMpi1	SIP_J163840_k2_exp_gmres_levelpmg	11/04/2021 18:08:08	b976d734...
#3: LinslvPerf_ConstPoissonMpi1	SIP_J20480_k5_exp_gmres_levelpmg	11/04/2021 18:09:39	493fc562...
#4: LinslvPerf_ConstPoissonMpi1	SIP_J20480_k3_classic_pardiso	11/04/2021 18:04:55	7963f80d...
#5: LinslvPerf_ConstPoissonMpi1	SIP_J69120_k2_classic_pardiso	11/04/2021 18:04:27	a4a94934...
#6: LinslvPerf_ConstPoissonMpi1	SIP_J69120_k2_exp_Kcycle_schwarz	11/04/2021 18:05:50	e3f9e9a3...
#7: LinslvPerf_ConstPoissonMpi1	SIP_J69120_k3_exp_gmres_levelpmg	11/04/2021 18:08:57	372e3ccb...
#8: LinslvPerf_ConstPoissonMpi1	SIP_J69120_k2_exp_gmres_levelpmg	11/04/2021 18:07:55	36bd8a75...
#9: LinslvPerf_ConstPoissonMpi1	SIP_J20480_k3_exp_Kcycle_schwarz	11/04/2021 18:06:32	69239995...
#10: LinslvPerf_ConstPoissonMpi1	S

In [27]:
var NoSuccess = wmg.AllJobs.Values.Where(job => job.Status != JobStatus.FinishedSuccessful);
NoSuccess

#0: SIP_J163840_k3_exp_Kcycle_schwarz: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#1: SIP_J20480_k5_exp_Kcycle_schwarz: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#2: SIP_J69120_k5_exp_Kcycle_schwarz: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#3: SIP_J163840_k3_exp_gmres_levelpmg: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#4: SIP_J69120_k5_exp_gmres_levelpmg: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)


In [ ]:
NUnit.Framework.Assert.Zero(NoSuccess.Count(), "Some Jobs Failed");

In [35]:
//foreach(Job jF in NoSuccess)
//   jF.DeleteOldDeploymentsAndSessions();

In [28]:
var FailedSessions = wmg.Sessions.Where(Si => Si.SuccessfulTermination == false);
FailedSessions

In [ ]:
NUnit.Framework.Assert.Zero(FailedSessions.Count(), "Some Sessions did not derminate successfully.");

List Output of some job:

In [ ]:
wmg.AllJobs.First().Value.Stdout

In [39]:
NoSuccess.Last().LatestDeployment.DeploymentDirectory.Name

LinslvPerf_ConstPoissonMpi1-ipPoisson2021Nov04_122801

## Create Table for Post-Processing

In [ ]:
// evaluators to add additional columns to the session table
static class AddCols {
    static public object XdgMatrixAssembly_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("MatrixAssembly");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Aggregation_basis_init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Aggregation_basis_init");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Solver_Init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Solver_Init");
        var nd  = ndS.ElementAt(0);
        //Console.WriteLine("Number of nodes: " + ndS.Count() + " cc " + nd.CallCount );
        return nd.TimeSpentInMethod.TotalSeconds / nd.CallCount;
    }
    static public object Solver_Run_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Solver_Run");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object NoOfCores(ISessionInfo SI){
        return SI.GetProfiling().Length;
        }
}

In [ ]:
wmg.AdditionalSessionTableColums.Clear();
wmg.AdditionalSessionTableColums.Add("MatrixAssembly", AddCols.XdgMatrixAssembly_time);
wmg.AdditionalSessionTableColums.Add("Aggregation_basis_init_time", AddCols.Aggregation_basis_init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Init_time", AddCols.Solver_Init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Run_time", AddCols.Solver_Run_time);
wmg.AdditionalSessionTableColums.Add("NoOfCores", AddCols.NoOfCores);

In [ ]:
var FullSessTab = wmg.SessionTable;

// The Session column can't be serialized,
// we have to remove it
List<string> AllCols = FullSessTab.GetColumnNames().ToList();
AllCols.Remove("Session");
FullSessTab = FullSessTab.ExtractColumns(AllCols.ToArray());

In [ ]:
//foreach(var s in AllCols) Console.WriteLine(s);

In [ ]:
//foreach(var s in wmg.Sessions[0].KeysAndQueries.Keys) Console.WriteLine(s + " \t = " + wmg.Sessions[0].KeysAndQueries[s]);

In [ ]:
var SubTab = FullSessTab.ExtractColumns(
    "SessionName","DGdegree:T", "Grid:NoOfCells", "LinearSolver.SolverCode", "DOFs", "MatrixAssembly",
    "Aggregation_basis_init_time", "Solver_Init_time", "Solver_Run_time", "NoIter");

In [ ]:
SubTab.Print();

In [ ]:
// Filename
var now               = DateTime.Now;
FullSessTab.TableName = wmg.CurrentProject + "_" + now.Year + "-" + now.Month + "-" + now.Day;
string docpath        = FullSessTab.TableName + ".json";
SubTab.SaveToFile(wmg.CurrentProject + ".json");
SubTab.ToCSVFile(wmg.CurrentProject + ".csv");